# Import and store NNDC deposition data
## First Objective

In [11]:
# Import relevent libraries
import requests
import json

In [3]:
# Constant variables
BASE_URL = "https://www.nndc.bnl.gov/"
JSON_FILE_PATH = "ensdf_output.json"

In [4]:
# Function to get the corresponding ENSDF file url by sending a post request to the 'decay_radiaton_url'
# All the other payload parameters are set to default, except for the "nuc"
# nucleus can be pass through the function parameters
def get_ENSDFfile_url(nucleus):

    decay_radiaton_url = BASE_URL + "nudat3/dec_searchi.jsp"

    payload = {
        "spnuc": "name",
        "nuc": nucleus,
        "z": "",
        "a": "",
        "n": "",
        "zmin": "",
        "zmax": "",
        "amin": "",
        "amax": "",
        "nmin": "",
        "nmax": "",
        "evenz": "any",
        "evena": "any",
        "evenn": "any",
        "tled": "disabled",
        "tlmin": "0",
        "utlow": "FS",
        "tlmax": "1E10",
        "utupp": "GY",
        "dmed": "disabled",
        "dmn": "ANY",
        "rted": "disabled",
        "rtn": "ANY",
        "reed": "disabled",
        "remin": "0",
        "remax": "10000",
        "ried": "disabled",
        "rimin": "0",
        "rimax": "200",
        "ord": "zate",
        "out": "wp",
        "unc": "nds",
        "sub": "S",
    }

    response = requests.post(decay_radiaton_url, data=payload)

    if response.status_code == 200:
        html_string = response.text

        try:
            # Extract the url from the response body
            ensdf_url = (
                html_string.split("<a href=")[2]
                .split("""class="zc" target=new2""")[0]
                .strip()
                .replace('"', "")
            )
            return ensdf_url
        except Exception as e:
            print("ERROR:", e)

    else:
        print("ERROR:", response.reason)
        return None

In [5]:
# Function to send a get request to the url where the ENSDF file is stored
# and obtain the ENSDF file from the response body
def get_ENSDF_file(ensdf_file_url):
    url = BASE_URL + "nudat3/" + ensdf_file_url.replace(" ", "%20") # Encode the URL in to the relevent format

    response = requests.get(url)
    if response.status_code == 200:
        html_string = response.text

        try:
            ensdf_file = html_string.split("<pre>")[1].split("</pre>")[0].strip() # Extract the ENSDF file from the response body
            return ensdf_file

        except Exception as e:
            print("ERROR:", e)

    else:
        print("ERROR:", response.reason)
        return None

In [6]:
# Function to send a post request to the radlist url with the corresponding payload
# and obtain the readable ENSDF file with the given format
# 'output_format' can be 'html','json','csv' or 'tsv'
def ensdf_to_readalbe(ensdf_file, output_format):
    radlist_url = BASE_URL + "radlist/radlist2.jsp"

    ensdf_file = " " + ensdf_file

    payload = {"mydata": ensdf_file, "format": output_format, "submit": "Submit"}

    response = requests.post(radlist_url, data=payload)

    if response.status_code == 200:
        json_object = json.dumps(response.json(), indent=4)
        print(json_object)

        with open(JSON_FILE_PATH, "w") as f:
            f.write(json_object)

    else:
        print("ERROR:", response.reason)

In [7]:
nucleus = "Co56"
ENSDF_file_url = get_ENSDFfile_url(nucleus)

In [8]:
print(ENSDF_file_url)

getdecaydataset.jsp?nucleus=56FE&dsid=56co ec decay


In [9]:
if ENSDF_file_url is not None:
    ENSDF_file = get_ENSDF_file(ENSDF_file_url)
    print(ENSDF_file)

56FE    56CO EC DECAY                 1990ME15,1988WA26,2008Dr0411NDS    201105
 56FE  H TYP=FUL$AUT=HUO JUNDE, HUO SU, YANG DONG$CIT=NDS 112, 1513 (2011)$
 56FE2 H CUT=29-Oct-2009$
 56FE c  Sources produced by {+56}Fe(p,n) and {+55}Mn(|a,3n).
 56FE2c  Production by {+54}Fe(|a,pn) and {+56}Fe(|a,p3n) (1996La20).
 56FE c  {+56}Co |e decay has been recommended as calibration standards for E|g
 56FE2c  and I|g see 1983LoZV, 1990Me15, 1990Ni03, 1990Tr06, 1991BaZS, 1999He10.
 56FE c  1990Me15: measured E|g, I|g for multi |g-ray calibration standards.
 56FE c  1988Wa26: measured E|g.
 56FE c  1986Br01: measured |g(|q) from oriented nuclei.
 56FE c  1984Oh03: measured |g(|q) from oriented nuclei.
 56FE c  1980St20: measured E|g, I|g, |g|g(|q), |g|g-coin with Ge(Li).
 56FE2c  Recalculated |a(K)exp from 1965Pe18.
 56FE c  1980Sh28: measured E|g, I|g, |g|g-coin and |g|g(|q).
 56FE c  1992ScZZ: measured |g-ray emission probabilities.
 56FE c  2008Dr04: measured I|g with HPGe.
 56FE c  2005MaZS: e

In [10]:
if ENSDF_file is not None:
    ensdf_to_readalbe(ENSDF_file, "json")

{
    "parent": {
        "a": "56",
        "decayMode": {
            "name": "EC",
            "uncertainty": "",
            "value": "100"
        },
        "level": {
            "jpis": "4+",
            "energy": {
                "uncertainty": "",
                "value": "0.0"
            }
        },
        "z": "27",
        "halflife": {
            "inSeconds": "6673190.4",
            "units": "D",
            "uncertainty": "26",
            "value": "77.236"
        },
        "qValue": {
            "uncertainty": "20",
            "value": "4566.0"
        },
        "n": "29",
        "element": "Co"
    },
    "radiation": {
        "betaPlusAV": [
            {
                "dose": "0.121",
                "radIntensityUncertainty": "14",
                "radIntensity": "19.7",
                "energyUncertainty": "6",
                "subType": "",
                "epEnergyUncertainty": "",
                "type": "bp av",
                "energy": "6.1E+2"